In [24]:
# based on tutorial provided here: /Users/tabaneslami/Desktop/git/Deep-Learning/LLM/Hugging_Face/basics.ipynb

In [25]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = #"your_LANGCHAIN_API_KEY"
os.environ['OPENAI_API_KEY'] = #"OPENAI_API_KEY"



In [26]:
#! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

In [27]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [28]:
# Load Documents
loader = WebBaseLoader(
    web_paths=("https://pcdslab.github.io/team/Taban-Eslami",),
    # bs_kwargs=dict(
    #     parse_only=bs4.SoupStrainer(
    #         class_=("post-content", "post-title", "post-header")
    #     )
   # ),
)
docs = loader.load()

In [29]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [30]:
# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [31]:
# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("who is taban?")

'Taban Eslami is a PhD student with a research focus on developing machine learning and HPC algorithms for the analysis of big fMRI data. He previously held the position of Senior Machine Learning Engineer at ZEISS.'

In [32]:
## without using RAG
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
rag_chain = (
    llm
)

# Question
rag_chain.invoke("who is taban eslami?")

AIMessage(content='There is limited information available about Taban Eslami. It is possible that they are a private individual or a relatively unknown public figure. Without more context, it is difficult to provide a specific answer.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 14, 'total_tokens': 55, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b855b22f-c27c-41b3-808c-4fc0342f302a-0', usage_metadata={'input_tokens': 14, 'output_tokens': 41, 'total_tokens': 55})